In [1]:
%load_ext dotenv
%dotenv

In [35]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.memory import ConversationSummaryMemory
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, chain
from operator import itemgetter

In [3]:
TEMPLATE = '''
The following is the friendly conversation between Human and AI.
The AI is talkative and provide lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully said it does not know.

Current conversation:
{message_log}

Human:
{question}

AI:
'''

prompt_template = PromptTemplate.from_template(template=TEMPLATE)

In [9]:
chat = ChatOpenAI(model_name = 'gpt-4',
                  model_kwargs={'seed':365},
                  temperature = 0,
                  max_tokens = 100)

/Users/aungkaungpyaepaing/PycharmProjects/LangChain/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3610: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if await self.run_code(code, result, async_=asy):


In [30]:
chat_memory = ConversationSummaryMemory(llm = ChatOpenAI(),
                                        memory_key = 'message_log')

In [10]:
question = "Can you give me an interesting face that i probably didn't know about?"

In [18]:
dictionary_output = RunnablePassthrough.assign(
    message_log = RunnableLambda(chat_memory.load_memory_variables) |
                  itemgetter('message_log')).invoke(
    {'question': question})

In [20]:
prompt_value_output = prompt_template.invoke(dictionary_output)

In [21]:
ai_message_output = chat.invoke(prompt_value_output)

In [26]:
response = StrOutputParser().invoke(ai_message_output)

In [27]:
chat_memory.save_context(inputs = {'input':question},
                         outputs = {'output':response})

In [29]:
chat_memory.load_memory_variables({})

{'message_log': 'The human asks if the AI can provide an interesting fact that they might not know about. The AI shares that the human face can make over 7,000 different expressions due to the intricate structure of its 43 muscles, which can be independently manipulated by the nervous system to convey various emotions and reactions.'}

# Building Chain

1. RunnablePassthrough()
2. ChatPromptTemplate()
3. ChatOpenAI()
4. StrOutputParser()
5. Did it in Part2 LCEL 11

In [33]:
chain1 = (
    RunnablePassthrough.assign(
        message_log = RunnableLambda(chat_memory.load_memory_variables) |
        itemgetter('message_log'))
    | prompt_template
    | chat
    | StrOutputParser()
)

question = "Can you give me an interesting face that i probably didn't know about?"

response = chain1.invoke({'question':question})

chat_memory.save_context(inputs = {'input':question},
                         outputs = {'output':response})

response

'Sure, did you know that the human face is capable of making 7,000 different expressions? This is due to the complex structure of our facial muscles, which allow us to convey a wide range of emotions and reactions.'

# Using Chain Decorator

In [39]:
@chain
def memory_chain(question):
    chain1 = (
        RunnablePassthrough.assign(
            message_log = RunnableLambda(chat_memory.load_memory_variables) |
            itemgetter('message_log'))
        | prompt_template
        | chat
        | StrOutputParser()
    )

    chain1.get_graph().print_ascii()

    response = chain1.invoke({'question':question})

    chat_memory.save_context(inputs = {'input':question},
                             outputs = {'output':response})

    return response

In [40]:
memory_chain.invoke("Can you give me interesting face that i probably didn't know about?")

              +----------------------------+          
              | Parallel<message_log>Input |          
              +----------------------------+          
                    **             ***                
                 ***                  ***             
               **                        **           
+-----------------------+                  **         
| load_memory_variables |                   *         
+-----------------------+                   *         
            *                               *         
            *                               *         
            *                               *         
        +--------+                   +-------------+  
        | Lambda |                   | Passthrough |  
        +--------+**                 +-------------+  
                    **              **                
                      ***        ***                  
                         **    **                     
          

'Absolutely! Did you know that the human nose can remember up to 50,000 different scents? This is because the olfactory bulb, which starts in the nose and runs along the bottom of the brain, has direct connections to the two brain areas that are strongly implicated in emotion and memory: the amygdala and hippocampus. Interestingly, visual, auditory (sound), and tactile (touch) information do not pass through these brain areas, which could be why smell, more than any other'

In [41]:
memory_chain.invoke("Can you elaborate a bit more on this fact?")

              +----------------------------+          
              | Parallel<message_log>Input |          
              +----------------------------+          
                    **             ***                
                 ***                  ***             
               **                        **           
+-----------------------+                  **         
| load_memory_variables |                   *         
+-----------------------+                   *         
            *                               *         
            *                               *         
            *                               *         
        +--------+                   +-------------+  
        | Lambda |                   | Passthrough |  
        +--------+**                 +-------------+  
                    **              **                
                      ***        ***                  
                         **    **                     
          

"Absolutely! The human nose is quite an amazing organ. It's not just there for smelling, but it also plays a crucial role in our sense of taste and in forming memories.\n\nThe nose has around 400 types of scent receptors that can detect at least 1 trillion different odors. These receptors are directly connected to the olfactory bulb, which is the part of the brain that analyzes smells. The olfactory bulb is part of the limbic system, the emotional center of the brain. This"